![image.png](attachment:image.png)

# 1: Spark Applications and Basic Spark Concepts
[pyspark Doc](https://spark.apache.org/docs/2.4.5/api/python/index.html)

Spark provides two sets of APIs, *Structured APIs* and *low-level APIs*. The Structured APIs are designed to implement the business logic of Spark applications and they hide the Spark internals of the *low-level API*. So for us , as a ETL developer, the Structured APIs are the best starting point to dive into data processing with Spark.

Today's challenge is to write our first little Spark application to get to get a first impression of the *Structured APIs* like `DataFrame`,`Dataset`, *SQL Tables/Views* and *Structured Streaming* and to undertsand some basic concepts like lazy evaluation of transformations, and data processing actions.

![image-2.png](attachment:image-2.png)

![image.png](attachment:image.png)

The first questions, that comes to our mind is, how to start a Spark application?

## SparkSession
Starting a Spark application generates a Spark job which is controlled and mangaged by exactly one *driver process* and several *executor processes* running across the cluster nodes doing the actual computational work. The driver process is controlled by a`SparkSession` object, which is the entry point of any Spark application, so there is always a one-to-one relationship between SparkSession and Spark application.

So how can westart a Spark session?

Since we don't want to type in all the code line by line into the interactive console, our Spark application must create it's own `SparkSession`object. So every Spark application starts with something like this:

![image.png](attachment:image.png)

In [0]:
spark.version

## Spark UI
Spark provides an UI to monitor the status and progess of Spark jobs. It is available on port 4040 on the driver node in the Spark cluster. Since running Spark in local mode, i.e. all processes are running on our laptop, we can access the UI on http://localhost:4040.

After having executed the next code block in the Spark DataFrames section, the UI showed this to us.

<img src= "./screenshots/day-002/day-002_Spark_UI.jpeg">

Clicking on a jobname provides further details and us trics figures regarding the job execution icluding a graphical representation of the execution DAG (directed acyclic graph).

<img src= "./screenshots/day-002/day-002_Spark_UI_job_details.jpeg">


## Spark DataFrames
run hello world example which creates our first `DataFrame`, and similar to the interactive console, the starting point is the `SparkSession` object named *spark*.

![image.png](attachment:image.png)

In [0]:
ourFirstDataFrame = spark.range(100).toDF("number")
ourFirstDataFrame.show(10)

Spark `DataFrame` objects look quite similar to Pandas dataframes. In fact, we can easily transform a Spark `DataFrame` into a Pandas dataframe.

***But caution:*** This us thod should only be used if the resulting Pandas’s DataFrame is expected to be small, as all the data is loaded into the driver’s us mory.

In [0]:
pandasDF = ourFirstDataFrame.toPandas()
pandasDF.head(10)

By the way, tranforming data to JSON is also easy. Each row is turned into a JSON document as one element in the returned list.

In [0]:
# Spark Connect does not support toJSON(). Use pandas for JSON conversion instead.
import json
json_records = ourFirstDataFrame.toPandas().to_dict(orient="records")
# Display as JSON string (first 10 rows)
print(json.dumps(json_records[:10], indent=2))

Ok, back to topic. The main difference bewteen Spark and Pandas is, that Pandas dataframes reside on a single machine whereas a Spark `DataFrame`ìs an abstraction of the in-memory optimized low-level API *Resilient Distributed Dataset* (`RDD`), which is designed to be split up data into partitions which can be spread across a cluster of potentially thousends of nodes. 
Spark `DataFrame`objects have a surprising characteristic, they are immutable once they are created. So how can data processing works when data structures are written in stone?
## Lazy Evaluation, Transformations and Actions
The few lines of our code already demonstrate that the Structured API has a functional design. Since `DataFrame`objects are immutable, we have to use functions which read `DataFrame` objects as input, do some kind of data transformation and create a new `DataFrame`which again can be the input of another function to do further transformations and generating another `DataFrame`. So finally we can simply concatenate functions to create a sequence of transformations to get our desired data result.

![image.png](attachment:image.png)

Ok, let's do it. we want to see, if how it works. First, we want to read some data from CSV file into a dataframe. The file has a header line and we want Spark to derive the schema, i.e. name and type of the columns, from the file. Nevertheless we could also specify a schema explicitly instead of deriving it from file. Determining the schema processing time, instead of load time, is an example of the common *schema-on-read* design of Big Data architectures.

Important to keep in mind is, that the column types are not Python types (or Scala or Java types if we use another API languages). All language API commands are mapped to the Spark internal language *Catalyst* having its own types. That's why all API languages provide the same performance.

In [0]:
# --- Volume 配置 (假设与前述保持一致) ---
CATALOG_NAME = "workspace"
SCHEMA_NAME = "default"
VOLUME_NAME = "course_data"
# 根据你的 DATA_MAPPINGS，假设 flight-data 位于 BDA2_Data 中
VOLUME_SUBFOLDER = "BDA2_Data" 
FILE_NAME = "flight-data/2015-summary.csv" 

# 构造完整的 Volume 路径
file_path = f"/Volumes/{CATALOG_NAME}/{SCHEMA_NAME}/{VOLUME_NAME}/{VOLUME_SUBFOLDER}/{FILE_NAME}"

dfFlight = spark.read\
   .option("inferSchema", "true")\
   .option("header", "true")\
   .csv(file_path)

print("✅ DataFrame 已成功加载。")
dfFlight.printSchema()
dfFlight.show(5)

Running this code doesn't show any observable result to us . That looks strange on first view. Actually, Spark hasn't done anything yet, except for deriving the schema by reading a small sample of rows. This is because Spark applies *lazy evaluation* of *transformations*, i.e. no data is moved or processed until Spark is forced to by an *action*, e.g. by calling the `write()` function. 

By defining transformations, we just give Spark a set of rules describing how a given `Dataframe` should be logically transformed into a new `Dataframe`object. By calling an action, we give Spark the command to apply these transformations, process the data and provideusthe results.

The reason for the lazy evaluation approach is, that Spark first wants to know the whole story about *what* should be done effectively before it tries to determine an efficient way *how* to do this. Therefore Spark first compiles all transformations into a **logical** directed acyclic graph (DAG), than analyses this DAG, applies optimizations (e.g. predicate push-down to datasources) whenever possible and splits up the optimized **physical** DAG into stages and parallelised tasks of `RDD` manipulations before starting to execute them.

![image.png](attachment:image.png)

Important to note is, tha `DataFrame`objects are kept in us mory when ever possible. In contrast to MapReduce, Spark tries to avoid writing intermediate results (i.e. `DataFrame` objects) to disk by *piplining* consecutive in-memory transformations, to gain better performance.

This piplining is only possible for *narrow* transformations. These are transformations where each input partition contributes only to one output partition or where the transormation can be applied partion by partition, so the partitions can be processed locally on the same cluster node. Simple row-based filter rules or commutative operations like summing up values, are common examples of narrow transformations.

On the other hand, in a *wide* transformation input partitions contribute to multiple output partitions, so data needs to be shuffled across cluster nodes. Sorting and average calculation are common wide transformations across multiple partitions. During shuffling Spark writes results to disk, so wide transformations are not performed in-memory.


Ok, now we want to see some action and Spark to showusthe first 10 lines in our data file.

In [0]:
dfFlight.show(10)

Now we triggered actual data processing so we can see the results. Next to showing data, there are two other types of actions: writing output data, e.g. to file and actions to collect data to native objects in the respercitve language.

we can even combine transformations and actions in one single command.

In [0]:
# --- Volume 配置 (假设与前述保持一致) ---
CATALOG_NAME = "workspace"
SCHEMA_NAME = "default"
VOLUME_NAME = "course_data"

# 假设这个文件在 BDA2 文件夹中 (请根据你的 GitHub 仓库结构确认)
VOLUME_SUBFOLDER = "BDA2_Data" 
FILE_NAME = "flight-data/2015-summary.csv" 

# 构造完整的 Volume 路径
file_path = f"/Volumes/{CATALOG_NAME}/{SCHEMA_NAME}/{VOLUME_NAME}/{VOLUME_SUBFOLDER}/{FILE_NAME}"

print(f"尝试读取路径: {file_path}")

# 修正后的 Spark 读取代码

dfFlight = spark.read\
   .option("inferSchema", "true")\
   .option("header", "true")\
   .csv(file_path) # <--- 使用了构造好的绝对路径
   
dfFlight.show(10)

print("✅ DataFrame 已成功加载并显示前 10 行。")

# 再次读取并显示前 10 行（使用绝对路径）
spark.read\
   .option("inferSchema", "true")\
   .option("header", "true")\
   .csv(file_path)\
   .show(10)

Obviously we can split up any sequence of transformations by asigning the intermedate `Dataframe` object to a variable and have a look into the intermediate results by calling the`show()` function on that `Dataframe`. This is a very nice feature foruswhen we need to debug complex analytical queries or ETL jobs which compile severeal subqueries together. If our subqueries provide the expected result, the bug must reside in the remaining part of our transormation logic so we can focus our analysis on that area.
## Query Explain Plans
So we've learned so far, that we just need to define the business logic by concatinating transformation functions and Spark does the optimisation for us . Fortunately Spark givesusinsight, how it will perform our query by calling the `explain()` function.

we want to see, how Spark would **physically** execute the sorting, which is a wide transformation. Each step in the explain plan actually generate a new `DataFrame`.

In [0]:
dfFlight.sort("count").show()

Reading the explain plan from bottom upwards, it tells us , that first, Spark performs a file scan and than range partitioning is applied shuffling the data over 200 output partitions by default to sort the data. 

In [0]:
spark.conf.get("spark.sql.shuffle.partitions")

Since we are  running in local mode on a single machine, it might be better to limit the number of partitions to 5. we can do this be chaging the configuration of the `SparkSession` object *spark*.

In [0]:
spark.conf.set("spark.sql.shuffle.partitions", "5")

The explain plan confirms, that our configuration change has the desired effect.

In [0]:
dfFlight.sort("count").explain()

The `explain()` function can helpusfiguring out how flexible we can chain up functions which define `DataFrame` to `DataFrame` transformations. For example, is it relevant for the query execution, whether we filter before selecting or the other way around? 

In [0]:
from pyspark.sql import SparkSession
# 如果你已经在 Notebook 中定义了 spark session，可以跳过上面两行

# --- Volume 配置 (假设与前述保持一致) ---
CATALOG_NAME = "workspace"
SCHEMA_NAME = "default"
VOLUME_NAME = "course_data"

# 请根据你的实际部署情况修改 VOLUME_SUBFOLDER！
# 假设 retail-data 位于 BDA2_Data 文件夹内
VOLUME_SUBFOLDER = "BDA2_Data" 

# 构造完整的 Volume 路径到包含 CSV 文件的文件夹
file_path_pattern = f"/Volumes/{CATALOG_NAME}/{SCHEMA_NAME}/{VOLUME_NAME}/{VOLUME_SUBFOLDER}/retail-data/by-day/*.csv"

print(f"尝试读取的路径模式: {file_path_pattern}")

# 修正后的 Spark 读取代码
dfRetail = spark.read\
    .format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load(file_path_pattern) # <--- 使用了构造好的绝对路径模式

print("✅ DataFrame 已成功加载。")
dfRetail.printSchema()
dfRetail.show(5)


Can we get a performance benefit, when we filter very early?

In [0]:
from pyspark.sql.functions import col

dfRetail.where(col("InvoiceNo") != 536365)\
    .select("InvoiceNo", "Description")\
    .explain()

Or can we stick to the well-known SQL pattern: SELECT ... FROM ... WHERE?

In [0]:
dfRetail.select("InvoiceNo", "Description")\
    .where(col("InvoiceNo") != 536365)\
    .explain()

The execution plan is the same. Again Spark does the optimization in the background and performs the filter before the column projection. In this case, the functional PAI providesusmore flexibility than the strict SQL syntax.

## Spark SQL, Tables and Views
<a href="https://docs.databricks.com/spark/latest/spark-sql/language-manual/index.html">SQL Language Reference</a> provided by Databricks.

[abc](https://docs.databricks.com/spark/latest/spark-sql/language-manual/index.html)

we've been working with relational databases and SQL for many years. So we are  happy to notice, that Spark also speaks our languange. In fact, the Spark SQL API supports the ANSI SQL 2003 standard. we can turn a `Dataframe` into a table or view, which we can query with SQL. All we need to do is register a table/view on that `Dataframe`.

In [0]:
dfFlight.createOrReplaceTempView("flight_data_2015")

Now we can write our Spark query as we did it for long times in classical databases, and we will get exactly the same result, as doing it the functional way.

This example calculates the top 5 countries having the highest number of flight destinations in 2015. Obviously the most flights went to the US.

In [0]:
# finding the top five destination countries by SQL
from pyspark.sql.functions import max, desc
# transformation
maxSql = spark.sql("""SELECT DEST_COUNTRY_NAME, sum(count) as destination_total
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
ORDER BY sum(count) DESC
LIMIT 5""")
# action
maxSql.show()

The equivalent functional query looks like this:

In [0]:
#transformation
dfFlight\
   .groupBy("DEST_COUNTRY_NAME")\
   .sum("count")\
   .withColumnRenamed("sum(count)","destination_total")\
   .sort(desc("destination_total"))\
   .limit(5)\
   .show()

The functional version looks touseven more self-explaing the **logical** transformations. The story reading it line-by-line is: first the data is grouped by the destination countries. Than, for each group (partition) the number of flights are summed up which generates a new, derived column which is than renamed. Afterwards the results are sorted is descending order by the calculated column and the output is limited by the top 5 rows.

Fortunately the convenience of using Spark SQL API instead of functions does not have an negative performance impact. The **physical** execution explain plans of both versions are exactly the same.

In [0]:
# both transformartions compile to the same plan
maxSql.explain()

In [0]:
dfFlight\
   .groupBy("DEST_COUNTRY_NAME")\
   .sum("count")\
   .withColumnRenamed("sum(count)","destination_total")\
   .sort(desc("destination_total"))\
   .limit(5)\
   .explain()

Interesting to note is that the `sum()` aggregation involves two *hashAggregate* steps. Because `sum()` is a commutative operation Spark first calculates partial sums partition by partition which is a narrow transformation. Afterwards the aggregted, i.e. already reduced data is shuffled (*Exchange hashpartitioning*) to calculate the overall sum across all partitions. This is another example how Spark optimizes the query execution by first analyzing  all transformations befor starting data processing.
## Spark Datasets
Datasets are a type-safe version of DataFrames. Since Python is a dynamically taped language, they are not available in pyspark but they can be used in the Java and Scala API. Good to keep in mind, but we skip it for now, since we prefer Python.

## Structured Streaming
So far we did all data processing in batch mode, i.e. all data get's processed at once. Batch mode forcesusto wait, until all data we want to analyse is available. Stream processing on the other hand enablesusto process data incrementally as it arrives, so we can get insights faster.

Stream processing in Spark is very similar to data processing in batch mode. The following example will demonstrate this. As far as we can see so for now, this is because Spark stream processing is actually event-triggered mirco-batch-processing. 

Ok, let's have a closer look and start from scratch, i.e. creating a new `SparkSession`.

In [0]:
# import pyspark
# from pyspark.sql import SparkSession
from pyspark.sql.functions import window, column, col, sum

# spark = SparkSession.builder.getOrCreate()
# spark = SparkSession.builder.master("local").config("spark.driver.memory", "8g").getOrCreate()
spark.conf.set("spark.sql.shuffle.partitions", "5")

### Batch processing
This time, our data source is not a single file, instead the data is split into several files on a day-by-day basis. Nevertheless, infering the schema, the us ta data, works the same way. The only difference is the wildcard * in the filename to tell Spark, that we want to process all CSV files in the specified folder.

In [0]:
# --- Volume 配置 (假设与前述保持一致) ---
CATALOG_NAME = "workspace"
SCHEMA_NAME = "default"
VOLUME_NAME = "course_data"

# 请根据你的实际部署情况修改 VOLUME_SUBFOLDER！
# 假设 retail-data 位于 BDA2_Data 文件夹内
VOLUME_SUBFOLDER = "BDA2_Data" 

# 构造完整的 Volume 路径到包含 CSV 文件的文件夹
# 这里的路径是 Volume 路径 + 仓库内部的 data 路径 + 文件模式
file_path_pattern = f"/Volumes/{CATALOG_NAME}/{SCHEMA_NAME}/{VOLUME_NAME}/{VOLUME_SUBFOLDER}/retail-data/by-day/*.csv"

print(f"尝试读取的路径模式: {file_path_pattern}")

# 修正后的 Spark 读取代码
staticDF = spark.read\
    .format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load(file_path_pattern) # <--- 使用了构造好的绝对路径模式

print("✅ DataFrame 已成功加载。")
staticDF.printSchema()
staticDF.show(5)


This defines the transformation which  tells Spark how to create the source DataFrame.

As a retailer, we want to analyse how much money each customer is pending in our shops per hour in each 1 day time window.
So we add a further transformation on that `DataFrame`, which describes the business logic of our data analysis, and results to another `DataFrame`.

In [0]:
purchaseByCustomerPerHour = staticDF\
   .selectExpr("CustomerId", "(UnitPrice * Quantity) as total_cost", "InvoiceDate")\
   .groupBy("CustomerId", window("InvoiceDate", "1 day"))\
   .sum("total_cost")

To take a look at the first 10 rows of the result, we have to call the action `show()`.

In [0]:
purchaseByCustomerPerHour.show(10)

In batch mode, we are  actually processing the entire data history, i.e. all files at once, which can take quite a long time for large data sets. To make this faster, we can switch to stream processing.
### Stream processing


There are just two things, we have to do, to turn our batch processing into stream processing in Spark:
 - using `readStream()` instead of `read()`, and
 - defining a trigger refreshing the result after reading each input file

In the given example, a trigger get's fired after reading each file (*maxFilesPerTrigger* = 1). Since all files are already on our harddrive, Spark will actually refresh the results every few (milli-)seconds, so finally we are  quite close to realtime-processing in this demonstration.

The schema is the same, as for batch processing, so we are  re-using it from the *staticDF*.

In [0]:
# import findspark
# findspark.init()

# from pyspark.sql import SparkSession

# spark = SparkSession.builder \
#     .appName("YourAppName") \
#     .getOrCreate()

In [0]:
# --- Volume 配置 (假设与前述保持一致) ---
CATALOG_NAME = "workspace"
SCHEMA_NAME = "default"
VOLUME_NAME = "course_data"

# 请根据你的实际部署情况修改 VOLUME_SUBFOLDER！
# 假设 retail-data 位于 BDA2_Data 文件夹内
VOLUME_SUBFOLDER = "BDA2_Data" 

# 构造完整的 Volume 路径到包含 CSV 文件的文件夹
# 这里的路径是 Volume 路径 + 仓库内部的 data 路径 + 文件模式
file_path_pattern = f"/Volumes/{CATALOG_NAME}/{SCHEMA_NAME}/{VOLUME_NAME}/{VOLUME_SUBFOLDER}/retail-data/by-day/*.csv"

print(f"尝试读取流的路径模式: {file_path_pattern}")

# 修正后的 Spark Streaming 读取代码
streamingDF = spark.readStream\
   .schema(staticDF.schema)\
   .option("maxFilesPerTrigger", 1)\
   .format("csv")\
   .option("header", "true")\
   .load(file_path_pattern) # <--- 使用了构造好的绝对路径模式

print("✅ Streaming DataFrame 已成功定义。")

Let's check, if the Stream creation was successfully.

In [0]:
streamingDF.isStreaming

The transformation is still the same, but now it is applied on a stream instead of a DataFrame.

In [0]:
purchaseByCustomerPerHour = streamingDF\
   .selectExpr("CustomerId", "(UnitPrice * Quantity) as total_cost", "InvoiceDate")\
   .groupBy(col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
   .sum("total_cost")

In [0]:
import os

# Define the checkpoint folder path in Unity Catalog Volume
checkpoint_folder = "/Volumes/workspace/default/course_data/checkpoints/customer_purchases"

# Create the checkpoint folder if it does not exist
os.makedirs(checkpoint_folder, exist_ok=True)

# Start the stream
# Your stream starting code goes here

As we've learnd so far, That Spark evaluates lazly and nothing happens untill we call an action to initiate the stream processing. The action `writeStream` generates a table, which gets updated after each trigger event. Important to note is, **streaming tables are mutable** whereas `DataFrame`objectss **are immutable.**

Here we stream the results to our console using `format("console")`, to make it visible how the result table gets updated regularly. Using`format("memory")` would push the stream to an in-memory table so other stream processes could read it.

In [0]:
CHECKPOINT_PATH = "/Volumes/workspace/default/course_data/checkpoints/customer_purchases"

purchaseByCustomerPerHour\
   .writeStream\
   .format("console")\
   .queryName("customer_purchases")\
   .outputMode("complete")\
   .trigger(once=True)\
   .option("checkpointLocation", CHECKPOINT_PATH)\
   .start()